##Inicio

###Importamos desde Drive.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


###Importamos librerias.

In [2]:
import os
import pandas as pd
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from datetime import datetime, timedelta

###Acceso al .csv "ruta_de_acceso"

In [3]:
ruta_de_acceso = '/content/drive/MyDrive/Big Data/TP1'

archivos = os.listdir(ruta_de_acceso)
ruta_completa = []
for archivo in archivos:
  if archivo.endswith('.csv'):
    ruta_completa.append(os.path.join(ruta_de_acceso,archivo))

###Generamos un Data Frame con informacion ordenada.

In [4]:
# Título: Proyecto de Análisis de Datos de Ventas Mensuales
# Curso: Máquinas y Big Data
# Laboratorio: 1

# Crear un DataFrame vacío para almacenar los datos de ventas
df_ventas = []

# Concatenar los archivos de datos en el DataFrame df_ventas
df_ventas = pd.concat(map(pd.read_csv, ruta_completa), ignore_index=True)

# Transformar valores no numéricos en NaN en las columnas 'Cantidad Pedida' y 'Precio Unitario'
df_ventas['Cantidad Pedida'] = pd.to_numeric(df_ventas['Cantidad Pedida'], errors='coerce')  # El errors= 'coerce' transforma los valores no numéricos a NaN
df_ventas['Precio Unitario'] = pd.to_numeric(df_ventas['Precio Unitario'], errors='coerce')

# Eliminar filas con NaN en las columnas 'Cantidad Pedida' y 'Precio Unitario'
df_ventas = df_ventas.loc[~df_ventas['Cantidad Pedida'].isna()]
df_ventas = df_ventas.loc[~df_ventas['Precio Unitario'].isna()]

# Eliminar filas con NaN en cualquier otra columna
df_ventas = df_ventas.dropna()

# Convertir 'Cantidad Pedida' a tipo de dato entero y 'Precio Unitario' a tipo de dato decimal con 2 decimales
df_ventas['Cantidad Pedida'] = df_ventas['Cantidad Pedida'].astype(int)
df_ventas['Precio Unitario'] = df_ventas['Precio Unitario'].astype(float).round(decimals=2)

# Definir listas para ordenar los días de la semana y los meses
dias_ordenados = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
meses_ordenados = ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]

# Convertir la columna 'Fecha de Pedido' a formato datetime y crear una nueva columna 'Fecha Convertida'
df_ventas['Fecha Convertida'] = pd.to_datetime(df_ventas['Fecha de Pedido'])
df_ventas = df_ventas.drop("Fecha de Pedido", axis=1)

# Crear la columna 'nombreDia' que contiene el nombre del día de la semana en español
df_ventas['nombreDia'] = df_ventas['Fecha Convertida'].dt.day_name()
df_ventas['nombreDia'] = pd.Categorical(df_ventas['nombreDia'], dias_ordenados)
df_ventas['nombreDia'] = df_ventas['nombreDia'].map({
    "Monday": "Lunes",
    "Tuesday": "Martes",
    "Wednesday": "Miércoles",
    "Thursday": "Jueves",
    "Friday": "Viernes",
    "Saturday": "Sábado",
    "Sunday": "Domingo"})

# Crear la columna 'Dia' que contiene el número del día
df_ventas['Dia'] = df_ventas['Fecha Convertida'].dt.day

# Crear la columna 'Mes' que contiene el nombre del mes en español
df_ventas['Mes'] = df_ventas['Fecha Convertida'].dt.month_name()
df_ventas['Mes'] = pd.Categorical(df_ventas['Mes'], meses_ordenados)
df_ventas['Mes'] = df_ventas['Mes'].map({
    "January": "Enero",
    "February": "Febrero",
    "March": "Marzo",
    "April": "Abril",
    "May": "Mayo",
    "June": "Junio",
    "July": "Julio",
    "August": "Agosto",
    "September": "Septiembre",
    "October": "Octubre",
    "November": "Noviembre",
    "December": "Diciembre"})

# Crear la columna 'Hora' que contiene la hora de la 'Fecha Convertida'
df_ventas['Hora'] = df_ventas['Fecha Convertida'].dt.hour

# Calcular y agregar la columna 'Precio Total'
df_ventas['Precio Total'] = df_ventas['Cantidad Pedida'] * df_ventas['Precio Unitario']
df_ventas['Precio Total'] = df_ventas['Precio Total'].astype(float).round(decimals=2)

# Separar la columna 'Dirección de Envío' en 'Ciudad' y 'Estado'
df_ventas['Ciudad'] = df_ventas['Dirección de Envio'].apply(lambda x: x.split(',')[1]) + ', ' + df_ventas['Dirección de Envio'].apply(lambda x: x.split(',')[2].split(' ')[1])
df_ventas['Estado'] = df_ventas['Dirección de Envio'].apply(lambda x: x.split(',')[2].split(' ')[1])

# Convertir 'Ciudad' y 'Estado' a cadenas de texto
df_ventas['Ciudad'] = df_ventas['Ciudad'].astype(str)
df_ventas['Estado'] = df_ventas['Estado'].astype(str)

# Reordenar las columnas del DataFrame
df_ventas = df_ventas[['ID de Pedido', 'Producto', 'Cantidad Pedida', 'Precio Unitario', 'Precio Total', 'Fecha Convertida', 'nombreDia', 'Dia', 'Mes', 'Hora', 'Dirección de Envio', 'Ciudad', 'Estado']]

# Ordenar las filas del DataFrame por la columna 'Fecha Convertida'
df_ventas = df_ventas.sort_values('Fecha Convertida')

# Eliminar datos del año 2020 para analizar solo el año 2019
df_ventas.drop(df_ventas[(df_ventas['Fecha Convertida'] >= '2020-01-01')].index, inplace=True)

# Reindexar el DataFrame
df_ventas.reset_index(drop=True, inplace=True)

# Mostrar el DataFrame resultante
df_ventas


,ID de Pedido,Producto,Cantidad Pedida,Precio Unitario,Precio Total,Fecha Convertida,nombreDia,Dia,Mes,Hora,Dirección de Envio,Ciudad,Estado
0,147268,Wired Headphones,1,11.99,11.99,2019-01-01 03:07:00,Martes,1,Enero,3,"9 Lake St, New York City, NY 10001","New York City, NY",NY
1,148041,USB-C Charging Cable,1,11.95,11.95,2019-01-01 03:40:00,Martes,1,Enero,3,"760 Church St, San Francisco, CA 94016","San Francisco, CA",CA
2,149343,Apple Airpods Headphones,1,150.00,150.00,2019-01-01 04:56:00,Martes,1,Enero,4,"735 5th St, New York City, NY 10001","New York City, NY",NY
3,149964,AAA Batteries (4-pack),1,2.99,2.99,2019-01-01 05:53:00,Martes,1,Enero,5,"75 Jackson St, Dallas, TX 75001","Dallas, TX",TX
4,149350,USB-C Charging Cable,2,11.95,23.90,2019-01-01 06:03:00,Martes,1,Enero,6,"943 2nd St, Atlanta, GA 30301","Atlanta, GA",GA
...,...,...,...,...,...,...,...,...,...,...,...,...,...
185911,297481,AA Batteries (4-pack),1,3.84,3.84,2019-12-31 23:42:00,Martes,31,Diciembre,23,"82 Hill St, Dallas, TX 75001","Dallas, TX",TX
185912,298406,AAA Batteries (4-pack),2,2.99,5.98,2019-12-31 23:42:00,Martes,31,Diciembre,23,"30 Elm St, San Francisco, CA 94016","San Francisco, CA",CA
185913,317048,AAA Batteries (4-pack),2,2.99,5.98,2019-12-31 23:45:00,Martes,31,Diciembre,23,"743 Adams St, San Francisco, CA 94016","San Francisco, CA",CA
185914,309773,AAA Batteries (4-pack),1,2.99,2.99,2019-12-31 23:52:00,Martes,31,Diciembre,23,"60 Hickory St, Los Angeles, CA 90001","Los Angeles, CA",CA


##1)

###A) y B)

In [5]:
# Genero una tabla temporal con dos columnas: Mes y Cantidad Pedida
df_cantidadPedidaPorMes = df_ventas.groupby(['Mes'])['Cantidad Pedida'].sum().reset_index()

# Genero el gráfico de barras con un color específico
figCantidadPedidaPorMes = px.bar(df_cantidadPedidaPorMes,
                                 x="Mes",
                                 y="Cantidad Pedida")

# Diseño del gráfico
figCantidadPedidaPorMes.update_layout(
    title="Unidades vendidas por mes",
    title_x=0.5,
    xaxis=dict(title="Meses"),
    yaxis=dict(title="Unidades vendidas", side="left")
)

# Agregar etiquetas de texto con la cantidad de unidades en las barras
figCantidadPedidaPorMes.update_traces(text=df_cantidadPedidaPorMes['Cantidad Pedida'], textposition='outside')

# Cambio el color de las barras
figCantidadPedidaPorMes.update_traces(marker_color='steelblue')  # Cambia 'blue' al color deseado

# Muestra el gráfico
figCantidadPedidaPorMes.show()


###C)

In [6]:
# Filtro las filas para que solo contengan datos del año 2019.
df_ventas_2019 = df_ventas.loc[df_ventas['Fecha Convertida'] <= '2019-12-31']

# Agrupo las filas por mes.
df_ventas_mensual = df_ventas_2019.groupby('Mes')

# Calculo el total de ventas para cada mes.
df_ventas_mensual = df_ventas_mensual['Precio Total'].sum()

# Creo un nuevo dataframe con las columnas `Mes` y `Total de Ventas`.
df_ventas_mensual = pd.DataFrame(df_ventas_mensual).reset_index()

df_ventas_mensual

,Mes,Precio Total
0,Enero,1813586.44
1,Febrero,2202022.42
2,Marzo,2807100.38
3,Abril,3390670.24
4,Mayo,3152606.75
5,Junio,2577802.26
6,Julio,2647775.76
7,Agosto,2244467.88
8,Septiembre,2097560.13
9,Octubre,3736726.88


In [7]:
# Crear el gráfico de barras
fig = px.bar(df_ventas_mensual, x='Mes', y='Precio Total', title='Total de Ventas por Mes en 2019')

# Personalizar el diseño del gráfico
fig.update_layout(xaxis_title='Mes', yaxis_title='Total de Ventas')

# Agregar etiquetas de texto redondeadas en las barras con solo el número
fig.update_traces(text=df_ventas_mensual['Precio Total'].round(0).astype(int), textposition='outside')

# Mostrar el gráfico
fig.show()

##2)

###A)

In [8]:
# Lista de horas que deseas analizar
horas_analizadas = list(range(24))

# Crear un DataFrame para almacenar los datos de ventas por hora y mes
data = {'Hora': [], 'Mes': [], 'Unidades Vendidas': []}

for hora in horas_analizadas:
    for mes in df_ventas['Mes'].unique():
        ventas_hora_mes = df_ventas[(df_ventas['Mes'] == mes) & (df_ventas['Hora'] == hora)]
        total_unidades_vendidas = ventas_hora_mes['Cantidad Pedida'].sum()
        data['Hora'].append(hora)
        data['Mes'].append(mes)
        data['Unidades Vendidas'].append(total_unidades_vendidas)

# Crear un DataFrame a partir de los datos
df_ventas_hora_mes = pd.DataFrame(data)

# Crear un gráfico de líneas
fig = px.line(df_ventas_hora_mes, x='Mes', y='Unidades Vendidas', color='Hora',
              title='Evolución de Ventas por Hora del Día (Por Mes)',
              labels={'Mes': 'Mes', 'Unidades Vendidas': 'Unidades Vendidas'})

# Personalizar el diseño del gráfico
fig.update_xaxes(categoryorder='array', categoryarray=df_ventas['Mes'].unique())  # Ordenar los meses correctamente
fig.update_xaxes(tickvals=list(df_ventas['Mes'].unique()))  # Mostrar todos los meses en el eje X
fig.update_yaxes(title='Unidades Vendidas', side="left")

# Mostrar el gráfico
fig.show()

##3)

###A)Listo

In [9]:
df_mayoresVentasPorCiudad = df_ventas.groupby('Ciudad')['Cantidad Pedida'].sum().reset_index()

# Genero el gráfico de barras con una paleta de colores discreta
figCiudad = px.bar(df_mayoresVentasPorCiudad,
                   x="Ciudad",
                   y="Cantidad Pedida",
                   text="Cantidad Pedida",
                   color="Ciudad",  # Usa la columna 'Ciudad' para determinar los colores
                   color_discrete_sequence=px.colors.qualitative.Set1)  # Paleta de colores

# Personalizo el diseño del gráfico
figCiudad.update_layout(
    title="Cantidad de Ventas por Ciudad",
    title_x=0.5,
    xaxis=dict(title="Ciudades"),
    yaxis=dict(title="Unidades vendidas", side="left")
)

# Muestra el gráfico
figCiudad.show()

###B)Listo

In [10]:
# Agrupo por estado y cuenta la cantidad de ventas para cada estado
ventas_por_estado = df_ventas['Estado'].value_counts().reset_index()
ventas_por_estado.columns = ['Estado', 'Cantidad de Ventas']

# Crear el gráfico de barras con Plotly Express
fig = px.bar(ventas_por_estado, x='Estado', y='Cantidad de Ventas', title='Cantidad de Ventas por Estado')

# Personalizar el diseño del gráfico
fig.update_layout(xaxis_title='Estado', yaxis_title='Cantidad de Ventas')

# Agregar etiquetas de texto con el número de ventas a las barras
fig.update_traces(text=ventas_por_estado['Cantidad de Ventas'], textposition='outside')

# Mostrar el gráfico de barras con las unidades en las barras
fig.show()


In [11]:
# Agrupa por estado y cuenta la cantidad de ventas para cada estado
ventas_por_estado = df_ventas['Estado'].value_counts().reset_index()
ventas_por_estado.columns = ['Estado', 'Cantidad de Ventas']

# Crear el mapa coroplético con Plotly Express
figMapEstado = px.choropleth(ventas_por_estado,
                             locations='Estado',
                             locationmode='USA-states',
                             scope='usa',
                             color='Cantidad de Ventas',  # Utiliza la columna correcta
                             color_continuous_scale='YlGnBu')

figMapEstado.update_layout(
    title='Cantidad de Ventas por Estado',
    title_x=0.5)

# Agregar etiquetas de texto con el número de ventas a las áreas
figMapEstado.add_scattergeo(
    locations=ventas_por_estado['Estado'],
    locationmode='USA-states',
    text=ventas_por_estado['Cantidad de Ventas'],
    mode='text',
    textposition='top center'
)

figMapEstado.show()


###C)Listo

In [12]:
# Crear el gráfico de líneas con Plotly Express
df_variacionVentasPorEstadoPorMes2 = df_ventas.groupby(['Mes','Estado'])['Cantidad Pedida'].sum().reset_index()
fig_line = px.line(df_variacionVentasPorEstadoPorMes2,
                   x="Mes",
                   y="Cantidad Pedida",
                   color="Estado",
                   markers=True,
                   title="Variación de las Unidades Vendidas por Estado por Mes",
                   labels={"Mes": "Meses", "Cantidad Pedida": "Unidades vendidas"})

# Configurar los ejes y los títulos
fig_line.update_xaxes(title_standoff=0.5)
fig_line.update_yaxes(title="Unidades vendidas", side="left")

fig_line.show()


In [13]:
#El siguiente dataframe me devuelve las cantidades vendidas agrupadas por Mes y por Hora
df_variacionVentasPorEstadoPorMes2 = df_ventas.groupby(['Mes','Estado'])['Cantidad Pedida'].sum(list).reset_index()

fig = px.bar(df_variacionVentasPorEstadoPorMes2, x="Mes", y="Cantidad Pedida",
             color="Estado",
             barmode = 'group')

# Configuro los ejes y los títulos
fig.update_layout(
    title="Variación de las Unidades Vendidas en los Estados por Mes",
    title_x = 0.5,
    xaxis=dict(title="Meses"),  # Para que en el eje x se muestren todos los valores uso tickvals=Horas
    yaxis=dict(title="Unidades vendidas", side="left")
    )

fig.show()

##4)

###A)

In [14]:
# Agrupa por producto y suma la cantidad pedida para obtener los más vendidos
df_productoMasVendido = df_ventas.groupby('Producto')['Cantidad Pedida'].sum().reset_index()
df_productoMasVendido = df_productoMasVendido.sort_values('Cantidad Pedida', ascending=False)

# Crear el gráfico de barras con Plotly Express
fig = px.bar(df_productoMasVendido, x="Producto", y="Cantidad Pedida", color="Cantidad Pedida",
             barmode="group", text="Cantidad Pedida",
             color_continuous_scale='viridis')  # Cambia 'viridis' al color deseado

fig.update_layout(
    title="Cantidades Vendidas por Producto",
    title_x=0.5,
    xaxis=dict(title="Producto"),
    yaxis=dict(title="Unidades vendidas", side="left")
)

# Ocultar la leyenda de las barras
for trace in fig.data:
    trace.showlegend = False

fig.show()


In [15]:
# Agrupar por mes y producto, y calcular la cantidad total vendida de cada producto en cada mes
df_ventas_productos_mes = df_ventas.groupby(['Mes', 'Producto'])['Cantidad Pedida'].sum().reset_index()

# Encontrar el producto más vendido en cada mes
producto_mas_vendido_por_mes = df_ventas_productos_mes.loc[df_ventas_productos_mes.groupby('Mes')['Cantidad Pedida'].idxmax()]

# Crear el gráfico de barras
fig = px.bar(producto_mas_vendido_por_mes, x='Mes', y='Cantidad Pedida', text='Producto',
             title='Producto Más Vendido por Mes')

# Personalizar el diseño del gráfico
fig.update_layout(xaxis_title='Mes', yaxis_title='Cantidad Pedida', title_x=0.5)

# Mostrar el gráfico
fig.show()


###B)

In [16]:
# Agrupo por producto y obtengo el precio mínimo
df_precioProductos = df_ventas.groupby('Producto')['Precio Unitario'].min().reset_index()

# Genero el gráfico de barras
fig = px.bar(df_precioProductos, x="Producto", y="Precio Unitario", text="Precio Unitario")

fig.update_layout(
    title="Precio de los Productos",
    title_x=0.5,
    xaxis=dict(title="Producto"),
    yaxis=dict(title="Precio", side="left", tickprefix='$', tickformat="${:,.2f}"),
)

fig.show()


##5)

###A) y B)

In [17]:
# Agrupar por día de la semana y calcular las ventas promedio
ventas_por_dia_semana = df_ventas.groupby('nombreDia')['Precio Total'].mean().reset_index()

# Ordenar los días de la semana en orden
dias_ordenados = ["Lunes", "Martes", "Miércoles", "Jueves", "Viernes", "Sábado", "Domingo"]
ventas_por_dia_semana['nombreDia'] = pd.Categorical(ventas_por_dia_semana['nombreDia'], categories=dias_ordenados, ordered=True)

# Crear el gráfico de líneas
fig = px.line(ventas_por_dia_semana, x='nombreDia', y='Precio Total', title='Ventas Promedio por Día de la Semana')

# Personalizar el diseño del gráfico
fig.update_layout(xaxis_title='Día de la Semana', yaxis_title='Ventas Promedio', title_x=0.5)

# Mostrar el gráfico
fig.show()

In [18]:
# Agrupar el DataFrame por día de la semana y calcular la venta promedio
ventas_promedio_dias_semana = df_ventas.groupby('nombreDia')['Precio Total'].mean().reset_index()

# Ordenar los días de la semana de forma personalizada
dias_ordenados = ["Lunes", "Martes", "Miércoles", "Jueves", "Viernes", "Sábado", "Domingo"]
ventas_promedio_dias_semana['nombreDia'] = pd.Categorical(ventas_promedio_dias_semana['nombreDia'], categories=dias_ordenados, ordered=True)
ventas_promedio_dias_semana = ventas_promedio_dias_semana.sort_values(by='nombreDia')

# Crear el gráfico de barras con etiquetas de texto
fig = px.bar(ventas_promedio_dias_semana, x='nombreDia', y='Precio Total',
             title='Ventas Promedio por Día de la Semana',
             labels={'nombreDia': 'Día de la Semana', 'Precio Total': 'Venta Promedio'},
             color_discrete_sequence=["#007ACC"])  # Cambia el código de color a tu elección

# Agregar etiquetas de texto con las unidades en cada barra
fig.update_traces(text=ventas_promedio_dias_semana['Precio Total'].round(2), textposition='outside')

# Personalizar el diseño del gráfico
fig.update_layout(xaxis_title='Día de la Semana', yaxis_title='Venta Promedio', title_x=0.5)

# Mostrar el gráfico con etiquetas de unidades
fig.show()


In [19]:
# Agrupar el DataFrame por mes y día de la semana y calcular la suma de las ventas
ventas_por_mes_dia = df_ventas.groupby(['Mes', 'nombreDia'])['Precio Total'].sum().reset_index()

# Crear el gráfico de barras apiladas
fig = px.bar(ventas_por_mes_dia, x='Mes', y='Precio Total', color='nombreDia',
             title='Distribución de Ventas por Día de la Semana en Cada Mes',
             category_orders={"nombreDia": ["Lunes", "Martes", "Miércoles", "Jueves", "Viernes", "Sábado", "Domingo"]})

# Personalizar las etiquetas de texto en las barras
fig.update_traces(texttemplate='%{y:.2f}', textposition='outside')

# Personalizar el diseño del gráfico
fig.update_layout(xaxis_title='Mes', yaxis_title='Ventas Totales', title_x=0.5)

# Mostrar el gráfico con etiquetas de unidades
fig.show()


In [20]:
# Agrupar el DataFrame por mes y día de la semana y calcular la suma de las ventas
ventas_por_mes_dia = df_ventas.groupby(['Mes', 'nombreDia'])['Precio Total'].sum().reset_index()

# Ordenar los días de la semana de forma personalizada
dias_ordenados = ["Lunes", "Martes", "Miércoles", "Jueves", "Viernes", "Sábado", "Domingo"]
ventas_por_mes_dia['nombreDia'] = pd.Categorical(ventas_por_mes_dia['nombreDia'], categories=dias_ordenados, ordered=True)
ventas_por_mes_dia = ventas_por_mes_dia.sort_values(by=['Mes', 'nombreDia'])

# Crear el gráfico de líneas
fig = px.line(ventas_por_mes_dia, x='Mes', y='Precio Total', color='nombreDia',
              title='Distribución de Ventas por Día de la Semana en Cada Mes',
              category_orders={"nombreDia": dias_ordenados})

# Personalizar el diseño del gráfico
fig.update_layout(xaxis_title='Mes', yaxis_title='Ventas Totales', title_x=0.5)

# Mostrar el gráfico
fig.show()


###C)

In [21]:
# Definir los nombres de los días de la semana en español
nombreDiaES = ["Lunes", "Martes", "Miércoles", "Jueves", "Viernes", "Sábado", "Domingo"]

# Calcular la cantidad total de ventas
cantidadTotaldeVentas = df_ventas['Precio Total'].sum()

# Calcular la cantidad de ventas en días laborales (de lunes a viernes)
cantidadVentasDiasLaborales = df_ventas[df_ventas['nombreDia'].isin(nombreDiaES[:5])]['Precio Total'].sum()

# Calcular la cantidad de ventas en el fin de semana (sábado y domingo)
cantidadVentasFinDeSemana = df_ventas[df_ventas['nombreDia'].isin(nombreDiaES[5:7])]['Precio Total'].sum()

# Crear un DataFrame para la comparación de ventas
filas = [('Semana Laboral', cantidadVentasDiasLaborales),
         ('Fin de Semana', cantidadVentasFinDeSemana)]

df_comparacionVentas = pd.DataFrame(filas, columns=['Período', 'Precio Total'])

# Generar el gráfico de barras con una paleta de colores verdosos
fig = px.bar(df_comparacionVentas, x="Período", y="Precio Total", color="Precio Total",
             title="Comparación de Ventas entre Semana Laboral y Fin de Semana",
             labels={'Período': 'Período', 'Precio Total': 'Venta Total'},
             text=df_comparacionVentas['Precio Total'].map("${:,.2f}".format),
             color_discrete_sequence=["#4CAF50", "#8BC34A"])  # Paleta de colores verdosos

# Personalizar el diseño del gráfico
fig.update_layout(title_x=0.5, xaxis_title="Período", yaxis_title="Venta Total")

# Mostrar el gráfico con etiquetas de unidades
fig.show()


In [22]:
# Definir los nombres de los días de la semana en español
nombreDiaES = ["Lunes", "Martes", "Miércoles", "Jueves", "Viernes", "Sábado", "Domingo"]

# Calcular la cantidad total de ventas
cantidadTotaldeVentas = df_ventas['Precio Total'].sum()

# Calcular la cantidad de ventas en días laborales (de lunes a viernes)
cantidadVentasDiasLaborales = df_ventas[df_ventas['nombreDia'].isin(nombreDiaES[:5])]['Precio Total'].sum()

# Calcular la cantidad de ventas en el fin de semana (sábado y domingo)
cantidadVentasFinDeSemana = df_ventas[df_ventas['nombreDia'].isin(nombreDiaES[5:7])]['Precio Total'].sum()

# Crear un DataFrame para la comparación de ventas
filas = [('Semana Laboral', cantidadVentasDiasLaborales),
         ('Fin de Semana', cantidadVentasFinDeSemana)]

df_comparacionVentas = pd.DataFrame(filas, columns=['Período', 'Precio Total'])

# Calcular el porcentaje de ventas para cada período
df_comparacionVentas['Porcentaje'] = (df_comparacionVentas['Precio Total'] / cantidadTotaldeVentas) * 100

# Generar el gráfico de torta
fig = px.pie(df_comparacionVentas, names='Período', values='Porcentaje',
             title='Comparación de Ventas entre Semana Laboral y Fin de Semana')

# Personalizar el diseño del gráfico
fig.update_layout(title_x=0.5)

# Mostrar el gráfico de torta
fig.show()


##6)

###A) y B)

In [23]:
#Genero una tabla con 3 columnas.
df_analisisDiasFestivos = pd.DataFrame(columns = ['Dia Festivo', 'Fecha', 'Cantidad Pedida'])

diasFestivos = {"Día de las madres": datetime.strptime("12-05-2019", "%d-%m-%Y"),
                "Día de la Independencia": datetime.strptime("04-07-2019", "%d-%m-%Y"),
                "Día del Trabajo": datetime.strptime("02-09-2019", "%d-%m-%Y"),
                "Día de los Veteranos": datetime.strptime("11-11-2019", "%d-%m-%Y"),
                "Día de Acción de Gracias": datetime.strptime("23-11-2019", "%d-%m-%Y"),
                "Navidad" : datetime.strptime("24-12-2019", "%d-%m-%Y"),
                "Año Nuevo" :datetime.strptime("31-12-2019", "%d-%m-%Y")}

#por cada dia festivo, sumo la cantidad de pedidos en los 5 días anteriores
for diaFestivo in diasFestivos:
#  print(diaFestivo, diasFestivos[diaFestivo] )
  i=1
  fechaFestiva = diasFestivos[diaFestivo]
  for i in range(6):
    ventas = df_ventas[(df_ventas['Fecha Convertida'] > fechaFestiva - timedelta(1)) & (df_ventas['Fecha Convertida'] < fechaFestiva)]['Cantidad Pedida'].sum()
    fila = [diaFestivo, fechaFestiva , ventas]
    fechaFestiva = fechaFestiva - timedelta(1)
    df_analisisDiasFestivos.loc[len(df_analisisDiasFestivos)] = fila

#Ordeno las Filas por fecha
df_analisisDiasFestivos = df_analisisDiasFestivos.sort_values('Fecha')
df_analisisDiasFestivos = df_analisisDiasFestivos.reset_index(drop = True)

figDiasFestivos = make_subplots(rows=3, cols=3, subplot_titles=df_analisisDiasFestivos['Dia Festivo'].unique())

figDiasFestivos.add_bar(x=df_analisisDiasFestivos['Fecha'][0:6], y=df_analisisDiasFestivos['Cantidad Pedida'][0:6], text = df_analisisDiasFestivos['Cantidad Pedida'][0:6], showlegend=False, col = 1, row = 1)
figDiasFestivos.add_bar(x=df_analisisDiasFestivos['Fecha'][6:12], y=df_analisisDiasFestivos['Cantidad Pedida'][6:12], text = df_analisisDiasFestivos['Cantidad Pedida'][6:12], showlegend=False,col = 2, row = 1)
figDiasFestivos.add_bar(x=df_analisisDiasFestivos['Fecha'][12:18], y=df_analisisDiasFestivos['Cantidad Pedida'][12:18], text = df_analisisDiasFestivos['Cantidad Pedida'][12:18], showlegend=False,col = 3, row = 1)
figDiasFestivos.add_bar(x=df_analisisDiasFestivos['Fecha'][18:24], y=df_analisisDiasFestivos['Cantidad Pedida'][18:24], text = df_analisisDiasFestivos['Cantidad Pedida'][18:24], showlegend=False,col = 1, row = 2)
figDiasFestivos.add_bar(x=df_analisisDiasFestivos['Fecha'][24:30], y=df_analisisDiasFestivos['Cantidad Pedida'][24:30], text = df_analisisDiasFestivos['Cantidad Pedida'][24:30], showlegend=False, col = 2, row = 2)
figDiasFestivos.add_bar(x=df_analisisDiasFestivos['Fecha'][30:36], y=df_analisisDiasFestivos['Cantidad Pedida'][30:36], text = df_analisisDiasFestivos['Cantidad Pedida'][30:36], showlegend=False, col = 3, row = 2)
figDiasFestivos.add_bar(x=df_analisisDiasFestivos['Fecha'][36:42], y=df_analisisDiasFestivos['Cantidad Pedida'][36:42], text = df_analisisDiasFestivos['Cantidad Pedida'][36:42], showlegend=False, col = 1, row = 3)

#calculo el promedio de ventas del mes para comparar con la fecha en cuestion
promedioMayo = df_ventas[(df_ventas['Fecha Convertida'] >= '2019/05/01') & (df_ventas['Fecha Convertida'] < '2019/06/01')]['Cantidad Pedida'].sum()/30
promedioMayo = promedioMayo.round(decimals = 2)

promedioJulio = df_ventas[(df_ventas['Fecha Convertida'] >= '2019/07/01') & (df_ventas['Fecha Convertida'] < '2019/08/01')]['Cantidad Pedida'].sum()/31
promedioJulio = promedioJulio.round(decimals = 2)

promedioAgosto = df_ventas[(df_ventas['Fecha Convertida'] >= '2019/08/01') & (df_ventas['Fecha Convertida'] < '2019/09/01')]['Cantidad Pedida'].sum()/31
promedioAgosto = promedioAgosto.round(decimals = 2)

promedioNoviembre = df_ventas[(df_ventas['Fecha Convertida'] >= '2019/11/01') & (df_ventas['Fecha Convertida'] < '2019/12/01')]['Cantidad Pedida'].sum()/30
promedioNoviembre = promedioNoviembre.round(decimals = 2)

promedioDiciembre = df_ventas[(df_ventas['Fecha Convertida'] >= '2019/12/01') & (df_ventas['Fecha Convertida'] < '2020/01/01')]['Cantidad Pedida'].sum()/31
promedioDiciembre = promedioDiciembre.round(decimals = 2)

figDiasFestivos.add_hline(y=promedioMayo, col = 1, row = 1, annotation_text = 'Promedio de Ventas',)
figDiasFestivos.add_hline(y=promedioJulio, col = 2, row = 1, annotation_text = 'Promedio de Ventas')
figDiasFestivos.add_hline(y=promedioAgosto, col = 3, row = 1, annotation_text = 'Promedio de Ventas')
figDiasFestivos.add_hline(y=promedioNoviembre, col = 1, row = 2, annotation_text = 'Promedio de Ventas')
figDiasFestivos.add_hline(y=promedioNoviembre, col = 2, row = 2, annotation_text = 'Promedio de Ventas')
figDiasFestivos.add_hline(y=promedioDiciembre, col = 3, row = 2, annotation_text = 'Promedio de Ventas')
figDiasFestivos.add_hline(y=promedioDiciembre, col = 1, row = 3, annotation_text = 'Promedio de Ventas')

#Actualizo el rango de los ejes Y para obtener mejor visualización
figDiasFestivos.update_yaxes(range=[500, 700], row=1, col=1)
figDiasFestivos.update_yaxes(range=[450, 600], row=1, col=2)
figDiasFestivos.update_yaxes(range=[350, 500], row=1, col=3)
figDiasFestivos.update_yaxes(range=[550, 800], row=2, col=1)
figDiasFestivos.update_yaxes(range=[550, 800], row=2, col=2)
figDiasFestivos.update_yaxes(range=[800, 1000], row=2, col=3)
figDiasFestivos.update_yaxes(range=[800, 1000], row=3, col=1)

figDiasFestivos.update_layout(
    title="Análisis de Ventas en (5) días cercanos a Fechas Festivas",
    title_x = 0.5)

figDiasFestivos.show()